In [5]:
import re
from typing import Dict, List

import chromadb
import pandas as pd
from tqdm import tqdm


def parse_qa_pairs(
    file_path: str, sample_percentage: float = 0.05
) -> List[Dict[str, str]]:
    """
    Parse Q&A pairs from the questions_output.txt file.
    Each Q&A pair becomes a separate document.
    Only processes a sample percentage of the data for faster execution.
    """
    import random

    qa_pairs = []

    with open(file_path, "r", encoding="utf-8") as file:
        content = file.read()

    # Split by double newlines to separate Q&A pairs
    pairs = content.split("\n\n")

    # Calculate how many pairs to sample
    total_pairs = len([p for p in pairs if p.strip()])
    sample_size = max(1, int(total_pairs * sample_percentage))

    print(f"Total Q&A pairs found: {total_pairs}")
    print(f"Sampling {sample_size} pairs ({sample_percentage*100:.1f}%)")

    # Filter out empty pairs first
    valid_pairs = [p for p in pairs if p.strip()]

    # Randomly sample pairs
    sampled_pairs = random.sample(valid_pairs, min(sample_size, len(valid_pairs)))

    # Process pairs with progress bar
    for i, pair in enumerate(tqdm(sampled_pairs, desc="Parsing Q&A pairs")):
        lines = pair.strip().split("\n")

        question = ""
        answer = ""

        for line in lines:
            if line.startswith("Question:"):
                question = line.replace("Question:", "").strip()
            elif line.startswith("Answer:"):
                answer = line.replace("Answer:", "").strip()

        if question and answer:
            qa_pairs.append({"question": question, "answer": answer, "id": f"qa_{i}"})

    return qa_pairs


def prepare_nutrition_qa_documents(
    file_path: str, sample_percentage: float = 0.05
) -> Dict:
    """
    Convert Q&A pairs into ChromaDB-ready documents.
    Each Q&A pair becomes a searchable document.
    """
    qa_pairs = parse_qa_pairs(file_path, sample_percentage)

    documents = []
    metadatas = []
    ids = []

    # Process Q&A pairs with progress bar
    for qa in tqdm(qa_pairs, desc="Preparing documents"):
        # Create rich document text for semantic search
        document_text = f"""
        Question: {qa['question']}
        Answer: {qa['answer']}
        
        This Q&A pair provides information about nutrition and health topics.
        """.strip()

        # Extract keywords from question for better searchability
        question_words = re.findall(r"\b\w+\b", qa["question"].lower())
        answer_words = re.findall(r"\b\w+\b", qa["answer"].lower())
        all_words = question_words + answer_words

        # Create metadata for filtering and exact lookups
        metadata = {
            "question": qa["question"],
            "answer": qa["answer"],
            "question_length": len(qa["question"]),
            "answer_length": len(qa["answer"]),
            "keywords": " ".join(set(all_words)),
            "has_question_mark": "?" in qa["question"],
            "topic": "nutrition_qa",
        }

        documents.append(document_text)
        metadatas.append(metadata)
        ids.append(qa["id"])

    return {"documents": documents, "metadatas": metadatas, "ids": ids}


def setup_nutrition_qa_chromadb(
    file_path: str,
    collection_name: str = "nutrition_qna",
    sample_percentage: float = 0.05,
):
    """
    Create and populate ChromaDB collection with nutrition Q&A data.
    """
    # Initialize ChromaDB
    client = chromadb.PersistentClient("../chroma")

    # Create collection (delete if exists)
    try:
        client.delete_collection(collection_name)
    except:
        pass

    collection = client.create_collection(
        name=collection_name,
        metadata={
            "description": "Nutrition Q&A database with questions and answers about nutrition and health"
        },
    )

    # Prepare documents
    data = prepare_nutrition_qa_documents(file_path, sample_percentage)

    # Add to ChromaDB with progress bar
    print("Adding documents to ChromaDB...")
    collection.add(
        documents=data["documents"], metadatas=data["metadatas"], ids=data["ids"]
    )

    print(
        f"Added {len(data['documents'])} Q&A pairs to ChromaDB collection '{collection_name}'"
    )
    return collection

In [6]:
# Set up the nutrition Q&A collection with 5% sample for faster execution

# The next line populates the db. It is commented out so you don't accidentally run it a it might run for tens of minutes
# collection = setup_nutrition_qa_chromadb("../data/questions_output.txt", "nutrition_qna", sample_percentage=0.05)

In [12]:
chroma_client = chromadb.PersistentClient("../chroma")
nutrition_qna = chroma_client.get_collection(name="nutrition_qna")
nutrition_qna.count()
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [8]:
# Test the setup with sample queries
chroma_client = chromadb.PersistentClient("../chroma")
nutrition_qna = chroma_client.get_collection(name="nutrition_qna")

# Test query 1: Search for malnutrition symptoms
print("=== Query: pregnancy ===")
results = nutrition_qna.query(query_texts=["pregnancy"], n_results=3)
for i, doc in enumerate(results["documents"][0]):
    print(f"Result {i+1}:")
    print(f"Answer: {results['documents'][0]}")
    print("\n")

print("\n" + "=" * 50 + "\n")

=== Query: pregnancy ===
Result 1:
Answer: ['Question: What are some possible physical changes that a pregnant woman could experience in the middle months of gestation?\n        Answer: During weeks 13-27, you may see an increase in weight and feel more hunger. Backaches might occur frequently, along with leg cramps and heartburn.\n\n        This Q&A pair provides information about nutrition and health topics.', 'Question: What health issues can make a pregnancy more challenging?\n        Answer: There are several medical conditions that can potentially increase the risk associated with Pregnancy. These include Anemia during this stage, Hypertensive Disorders related to gestation, Diabetes Mellitus coexisting with Pregnancy, Obesity during pregnancy, as well as Adolescent or Teenage pregnancies.\n\n        This Q&A pair provides information about nutrition and health topics.', "Question: What are some hormonal changes that take place in a mother's body during pregnancy?\n        Answer

In [10]:
from agents import Agent, Runner, function_tool, trace

In [13]:
@function_tool
def nutrition_qna_tool(query: str, n_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up nutrition information for specific keywords.

    Args:
        query: The question.
        n_results: The number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_qna.query(query_texts=[query], n_results=3)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    return "Nutrition Information:\n" + "\n".join(results)

In [14]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

In [21]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information and nutrtion advice    .
    You give concise answers.
    
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrtition_qna_tool first to see if there is an answer in the knowledge base.
    If are asked a question about calories, always use the calorie_lookup_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool, nutrition_qna_tool],
)

In [25]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "What are the best meal choices for pregnant women and how many calories do they have?",
    )
    print(result.final_output)

Here are concise guidelines and meal ideas for pregnant women, with approximate calorie ranges.

Best meal choices (balanced, nutrient-dense)
- Protein: lean meat, poultry, fish (low-mercury), eggs, dairy, beans, lentils, tofu.
- Whole grains: oats, quinoa, brown rice, whole-wheat bread or tortillas.
- Fruits and vegetables: aim for a colorful variety; include leafy greens for folate and iron.
- Healthy fats: avocados, nuts/seeds, olive oil, fatty fish (low-mercury) for DHA.
- Calcium and vitamin D: dairy or fortified alternatives; fortified plant milks.
- Iron-rich options: lean red meat (in moderation), beans, lentils, fortified cereals; pair with vitamin C-rich foods to enhance absorption.
- Folate: leafy greens, citrus, legumes; consider prenatal vitamins as advised by a clinician.
- Hydration: plenty of water; limit sugary drinks.

Approximate calorie guidance (additional needs vs. nonpregnant)
- 2nd trimester: about +300 to +340 kcal/day.
- 3rd trimester: about +430 to +450 kcal/